#### Загрузите данные ex9_movies.mat из файла.

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
DATA_PATH = '/home/ataleckij/Projects/university/mo/Data/Lab 9/'

In [3]:
files_names = os.listdir(DATA_PATH)
print(files_names)
files_paths = [os.path.join(DATA_PATH, name) for name in files_names]

['ex9_movies.mat', 'movie_ids.txt']


In [4]:
from scipy.io import loadmat

data0 = loadmat(files_paths[0])
Y, R, = data0['Y'], data0['R']
Y.shape, R.shape

((1682, 943), (1682, 943))

#### Выберите число признаков фильмов (n) для реализации алгоритма коллаборативной фильтрации.

In [27]:
n = 10

In [28]:
nU, nM = Y.shape

ETA = np.random.randn(nU, n)
X = np.random.randn(nM, n)

In [29]:
Y

array([[5, 4, 0, ..., 5, 0, 0],
       [3, 0, 0, ..., 0, 0, 5],
       [4, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

#### Реализуйте функцию стоимости для алгоритма.
#### Реализуйте функцию вычисления градиентов.
#### При реализации используйте векторизацию для ускорения процесса обучения.
#### Добавьте L2-регуляризацию в модель.

In [23]:
def loss(ETA, X, Y, R, lambda_):
    return 1/2 * ((ETA.dot(X.T) * R - Y) ** 2).sum() \
            + lambda_/2 * (ETA ** 2).sum() \
            + lambda_/2 * (X ** 2).sum()

In [24]:
def grad_tape_ETA(ETA, X, Y, R, lr, lambda_):
    new_ETA = ETA.copy()
    new_ETA -= lr * ((ETA.dot(X.T) * R - Y).dot(X) + lambda_ * ETA)
    return new_ETA, X

def grad_tape_X(ETA, X, Y, R, lr, lambda_):
    new_X = X.copy()
    new_X -= lr * ((ETA.dot(X.T) * R - Y).T.dot(ETA) + lambda_ * X)
    return ETA, new_X

In [10]:
ETA_, X_ = ETA, X
loss(ETA, X, Y, R, lambda_=0.05)

5826818.68019954

In [11]:
ETA_, X_ = grad_tape_ETA(ETA_, X_, Y, R, lr=0.001, lambda_=0.05)
ETA_, X_ = grad_tape_X(ETA_, X_, Y, R, lr=0.001, lambda_=0.05)
loss(ETA_, X_, Y, R, lambda_=0.05)

2392215.798709869

#### Обучите модель с помощью градиентного спуска или других методов оптимизации.

In [38]:
from tqdm import tqdm

def fit(ETA, X, eps, lambda_, lr, maxiter):
    ETA, X = ETA.copy(), X.copy()
    loss_value = loss(ETA, X, Y, R, lambda_=lambda_)
#     for _ in tqdm(range(maxiter)):
    for i in range(maxiter):
        ETA, X = grad_tape_ETA(ETA, X, Y, R, lr=lr, lambda_=lambda_)
        ETA, X = grad_tape_X(ETA, X, Y, R, lr=lr, lambda_=lambda_)
        new_loss_value = loss(ETA, X, Y, R, lambda_=lambda_)
        print(i, abs(loss_value - new_loss_value))
        if abs(loss_value - new_loss_value) <= eps:
            break
        loss_value = new_loss_value
    return ETA, X

In [53]:
ETA_, X_ = fit(ETA, X, eps=1e-5, lambda_=0.005, 
               lr=0.0005, maxiter=50_000)

0 160736.36820579716
1 95277.10333514377
2 65463.192774621304
3 51793.75948973431
4 47503.40028179414
5 49868.79932467989
6 56991.55485901609
7 65866.75785528659
8 71801.05720956449
9 70677.85201104125
10 62626.62016616401
11 51680.370357972395
12 41490.887057461834
13 33297.01238507993
14 26955.863388472266
15 22046.104240196175
16 18211.86352848049
17 15190.02241909827
18 12786.737356275844
19 10857.83309380745
20 9295.430656120967
21 8018.454725822259
22 6965.669675979705
23 6090.533492882838
24 5357.4079325455095
25 4738.779587194731
26 4213.225678208313
27 3763.922656573486
28 3377.548020080314
29 3043.466582734138
30 2753.123016657788
31 2499.584743464715
32 2277.1951621646294
33 2081.3084733266733
34 1908.0853283497563
35 1754.3341802038049
36 1617.3872362493275
37 1495.0028021324251
38 1385.2878956065979
39 1286.6365336043236
40 1197.6802167525166
41 1117.247965807197
42 1044.3338837856281
43 978.0706825254674
44 917.7079635718183
45 862.5943100974837
46 812.1624503817802
47 76

371 14.317576362875116
372 14.269222238493967
373 14.221192654447805
374 14.173484472703421
375 14.126094593309972
376 14.079019954537216
377 14.03225753170409
378 13.985804337396985
379 13.939657419781724
380 13.89381386309833
381 13.84827078627859
382 13.803025343113404
383 13.758074720964942
384 13.713416140861227
385 13.66904685676127
386 13.624964154994814
387 13.581165353782126
388 13.537647802753781
389 13.494408882688731
390 13.451446004728496
391 13.408756610064302
392 13.366338169529627
393 13.324188182974467
394 13.28230417924351
395 13.240683715303021
396 13.199324375898868
397 13.1582237737166
398 13.117379547933524
399 13.076789364946308
400 13.03645091708313
401 12.996361922865617
402 12.956520126288524
403 12.91692329657235
404 12.877569227624917
405 12.838455738317862
406 12.799580671118747
407 12.760941892658593
408 12.722537293113419
409 12.68436478546937
410 12.646422305981105
411 12.608707813233195
412 12.571219288103748
413 12.53395473321143
414 12.496912173286546

740 6.283411832726415
741 6.273143695878389
742 6.262902527068945
743 6.252688215798116
744 6.24250065212982
745 6.232339727059298
746 6.222205332327576
747 6.212097360112239
748 6.202015703776851
749 6.19196025679048
750 6.181930913840915
751 6.171927569954278
752 6.161950121088012
753 6.1519984636652225
754 6.1420724948329735
755 6.132172112527769
756 6.122297215184517
757 6.112447701932979
758 6.102623472554114
759 6.092824427556479
760 6.083050467830617
761 6.073301495191117
762 6.063577411834558
763 6.0538781207069405
764 6.044203525318153
765 6.034553529727418
766 6.024928038670623
767 6.015326957462094
768 6.005750191940024
769 5.996197648571979
770 5.986669234360306
771 5.977164856976742
772 5.967684424478648
773 5.958227845610963
774 5.948795029697067
775 5.939385886445962
776 5.930000326265144
777 5.920638260075066
778 5.911299599250924
779 5.901984255728166
780 5.892692141987936
781 5.883423171027971
782 5.8741772563626
783 5.864954311920883
784 5.855754252363113
785 5.84657

1106 3.750058231926232
1107 3.7454470188858977
1108 3.7408444956308813
1109 3.7362506403769657
1110 3.7316654312853643
1111 3.7270888466155156
1112 3.72252086471417
1113 3.717961463975371
1114 3.7134106227968005
1115 3.7088683199144725
1116 3.704334533707879
1117 3.699809242873016
1118 3.6952924262732267
1119 3.690784062611783
1120 3.6862841307483905
1121 3.6817926096227893
1122 3.677309478276584
1123 3.6728347156386008
1124 3.668368300939619
1125 3.663910213363124
1126 3.659460432081687
1127 3.655018936467968
1128 3.65058570581823
1129 3.64616071965429
1130 3.641743957403378
1131 3.6373353986818984
1132 3.632935022993479
1133 3.628542810198269
1134 3.6241587398544652
1135 3.619782791869511
1136 3.6154149460599
1137 3.611055182347627
1138 3.6067034807711025
1139 3.6023598212268553
1140 3.5980241839934024
1141 3.593696549120068
1142 3.589376896768954
1143 3.585065207324078
1144 3.5807614610966993
1145 3.576465638441732
1146 3.572177719732281
1147 3.567897685668868
1148 3.563625516584579

1456 2.5440000358394173
1457 2.5414455604404793
1458 2.5388949123189377
1459 2.536348083314806
1460 2.5338050653263053
1461 2.5312658503462444
1462 2.528730430331052
1463 2.5261987972189672
1464 2.52367094312649
1465 2.521146859991859
1466 2.518626539964316
1467 2.516109975069412
1468 2.5135971573727147
1469 2.511088079179899
1470 2.5085827324764978
1471 2.506081109513616
1472 2.503583202487789
1473 2.5010890036064666
1474 2.4985985051680473
1475 2.4961116994054464
1476 2.493628578569769
1477 2.4911491351376753
1478 2.488673361236579
1479 2.486201249426813
1480 2.4837327919558447
1481 2.48126798130761
1482 2.4788068098641816
1483 2.4763492701204086
1484 2.4738953545092954
1485 2.471445055649383
1486 2.468998365897278
1487 2.466555277875159
1488 2.4641157841724635
1489 2.461679877294955
1490 2.459247549984866
1491 2.45681879472977
1492 2.4543936043228314
1493 2.4519719712770893
1494 2.4495538884584676
1495 2.4471393484418513
1496 2.4447283440422325
1497 2.442320868041861
1498 2.43991691

...

48693 0.004721465567854466
48694 0.004721301542303991
48695 0.004721137509477558
48696 0.004720973505754955
48697 0.004720809502032353
48698 0.004720645461929962
48699 0.004720481512777042
48700 0.004720317534520291
48701 0.004720153548987582
48702 0.004719989581644768
48703 0.004719825610663975
48704 0.004719661723356694
48705 0.0047194977378239855
48706 0.004719333777757129
48707 0.0047191699013637844
48708 0.004719006010418525
48709 0.004718842094007414
48710 0.004718678152130451
48711 0.004718514293927001
48712 0.004718350424809614
48713 0.004718186584796058
48714 0.00471802269021282
48715 0.004717858817457454
48716 0.004717695010185707
48717 0.0047175311628961936
48718 0.004717367304692743
48719 0.0047172035338007845
48720 0.004717039686511271
48721 0.004716875919257291
48722 0.004716712101071607
48723 0.004716548341093585
48724 0.004716384533821838
48725 0.004716220784757752
48726 0.004716057035693666
48727 0.00471589328662958
48728 0.004715729530289536
48729 0.004715565788501408

48996 0.004672153809224255
48997 0.004671992421208415
48998 0.004671830898587359
48999 0.004671669488743646
49000 0.004671508071623975
49001 0.004671346654504305
49002 0.0046711852664884645
49003 0.004671023827540921
49004 0.004670862421335187
49005 0.004670701051509241
49006 0.0046705396816832945
49007 0.004670378279115539
49008 0.004670216967497254
49009 0.004670055583119392
49010 0.0046698942460352555
49011 0.00466973293441697
49012 0.004669571571866982
49013 0.004669410292990506
49014 0.004669248930440517
49015 0.004669087677029893
49016 0.00466892635813565
49017 0.0046687650683452375
49018 0.004668603807658656
49019 0.004668442550610052
49020 0.004668281279009534
49021 0.004668120032874867
49022 0.004667958783102222
49023 0.0046677975478814915
49024 0.004667636349040549
49025 0.00466747508107801
49026 0.0046673139186168555
49027 0.004667152705223998
49028 0.004666991499107098
49029 0.004666830303904135
49030 0.004666669188736705
49031 0.0046665079789818265
49032 0.0046663468201586

49299 0.004623619432095438
49300 0.004623460492439335
49301 0.004623301618266851
49302 0.004623142704076599
49303 0.004622983884473797
49304 0.004622824963007588
49305 0.004622666117938934
49306 0.00462250727287028
49307 0.004622348424163647
49308 0.004622189593646908
49309 0.004622030777682085
49310 0.004621871961717261
49311 0.004621713142114459
49312 0.0046215543370635714
49313 0.004621395542926621
49314 0.004621236752427649
49315 0.004621077994670486
49316 0.004620919211447472
49317 0.004620760453690309
49318 0.004620601703209104
49319 0.004620442963641835
49320 0.004620284242264461
49321 0.004620125502697192
49322 0.00461996680314769
49323 0.004619808056304464
49324 0.004619649374944856
49325 0.00461949068994727
49326 0.004619332034053514
49327 0.00461917327993433
49328 0.004619014671334298
49329 0.004618855997250648
49330 0.004618697330442956
49331 0.004618538710928988
49332 0.0046183800768631045
49333 0.004618221471901052
49334 0.004618062805093359
49335 0.004617904211045243
493

49604 0.004575541068334132
49605 0.004575384678901173
49606 0.004575228351313854
49607 0.004575071961880894
49608 0.004574915623379638
49609 0.004574759332172107
49610 0.004574603011860745
49611 0.004574446651531616
49612 0.004574290378513979
49613 0.0045741340836684685
49614 0.004573977799736895
49615 0.004573821475787554
49616 0.004573665257339599
49617 0.004573508933390258
49618 0.004573352740408154
49619 0.004573196460114559
49620 0.004573040241666604
49621 0.004572883965010988
49622 0.004572727812046651
49623 0.004572571557218907
49624 0.004572415360598825
49625 0.004572259127598954
49626 0.004572102949168766
49627 0.00457194682530826
49628 0.004571790595946368
49629 0.004571634442982031
49630 0.004571478275465779
49631 0.004571322169795167
49632 0.004571165998640936
49633 0.004571009863866493
49634 0.004570853758195881
49635 0.004570697605231544
49636 0.004570541517750826
49637 0.004570385426632129
49638 0.004570229350065347
49639 0.004570073248032713
49640 0.00456991717510391
49

49907 0.004528539615421323
49908 0.004528385718003847
49909 0.0045282318933459464
49910 0.004528077959548682
49911 0.004527924142166739
49912 0.004527770335698733
49913 0.004527616438281257
49914 0.004527462642727187
49915 0.004527308832621202
49916 0.004527155015239259
49917 0.004527001205133274
49918 0.0045268474459589925
49919 0.004526693624939071
49920 0.004526539829385001
49921 0.004526386095676571
49922 0.004526232325588353
49923 0.00452607857005205
49924 0.004525924832705641
49925 0.004525771106273169
49926 0.00452561736892676
49927 0.004525463609752478
49928 0.004525309948803624
49929 0.004525156211457215
49930 0.004525002535956446
49931 0.004524848853179719
49932 0.004524695174040971
49933 0.004524541494902223
49934 0.004524387823039433
49935 0.004524234143900685
49936 0.0045240805702633224
49937 0.004523926887486596
49938 0.004523773244727636
49939 0.0045236196492624
49940 0.0045234660065034404
49941 0.004523312414676184
49942 0.004523158833762864
49943 0.004523005231021671
4

#### Добавьте несколько оценок фильмов от себя. Файл movie_ids.txt содержит индексы каждого из фильмов.

In [42]:
with open(files_paths[1], 'rt', encoding='utf-8', errors='ignore') as f:
    films = [(int(film.split(' ')[0]), ' '.join(film.split(' ')[1:]))
             for film in f.read().split('\n')[:-1]]
    df_films = pd.DataFrame.from_records(films, columns=['film_index', 'film_name'])

In [43]:
i = 360
df_films.iloc[i:i+20]

,film_index,film_name
360,361,Incognito (1997)
361,362,Blues Brothers 2000 (1998)
362,363,Sudden Death (1995)
363,364,Ace Ventura: When Nature Calls (1995)
364,365,Powder (1995)
365,366,Dangerous Minds (1995)
366,367,Clueless (1995)
367,368,Bio-Dome (1996)
368,369,Black Sheep (1996)
369,370,Mary Reilly (1996)


#### Сделайте рекомендации для себя. Совпали ли они с реальностью?

In [ ]:
(1, 5), (27, 5), (50, 4), (96, 5) (373, 4)

In [55]:
def get_most_similar(film_index, X, n_best=5):
    film_vector = X[film_index - 1]
    similiarities = X.dot(film_vector) / (np.linalg.norm(X_, axis=1) * np.linalg.norm(film_vector))
    best_indexes = np.argsort(similiarities)[-(n_best + 1):][::-1][1:]
    print('best matches for: ' + df_films.iloc[film_index - 1].values.astype(str)[1])
    print(df_films.iloc[best_indexes].values)

In [56]:
get_most_similar(50, X_)

best matches for: Star Wars (1977)
[[550 'Die Hard: With a Vengeance (1995)']
 [248 'Grosse Pointe Blank (1997)']
 [281 'River Wild, The (1994)']
 [778 'Don Juan DeMarco (1995)']
 [672 'Candyman (1992)']]


In [70]:
get_most_similar(1, X_)

best matches for: Toy Story (1995)
[[268 'Chasing Amy (1997)']
 [429 'Day the Earth Stood Still, The (1951)']
 [45 'Eat Drink Man Woman (1994)']
 [868 'Hearts and Minds (1996)']
 [622 'Swiss Family Robinson (1960)']]


#### Также обучите модель с помощью сингулярного разложения матриц. Отличаются ли полученные результаты?

In [61]:
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds

Y_sparce = csc_matrix(Y, dtype=float)
u, s, v = svds(Y_sparce, k=10)

In [69]:
get_most_similar(50, v.T)

best matches for: Star Wars (1977)
[[21 'Muppet Treasure Island (1996)']
 [276 'Leaving Las Vegas (1995)']
 [201 'Evil Dead II (1987)']
 [181 'Return of the Jedi (1983)']
 [303 "Ulee's Gold (1997)"]]


In [71]:
get_most_similar(1, v.T)

best matches for: Toy Story (1995)
[[883 'Telling Lies in America (1997)']
 [916 'Lost in Space (1998)']
 [268 'Chasing Amy (1997)']
 [158 "Weekend at Bernie's (1989)"]
 [62 'Stargate (1994)']]
